In [83]:
from gurobipy import *
import pandas as pd
import numpy as np
import scipy.sparse as sp
from copy import deepcopy
import itertools
from random import *
import timeit
import networkx as nx

In [116]:
m = 6
n = 6
gen  = n*m
edges = []
operators = n
#1) Input network file
links = 2*m*n-m-n; # number of links
link = dict()
for i in range(1,n*m+1):
        if (i % n != 0):
            if ( gen - i >= n):
                edges.extend([(i,i+1),(i+1,i),(i,i+m),(i+m,i)])
            else: 
                edges.extend([(i,i+1),(i+1,i)])
        else: 
            if ( gen - i >= n):
                edges.extend([(i,i+m),(i+m,i)])
            else: 
                break;  
tt = dict();
capacity = dict();
operator = dict();
failure_prob = dict();
edge_dict = dict()
fail = [];
prob = [];
lst_prob = [];
K = [];
demand = dict();
cost = dict();
count = 0;
G = nx.DiGraph()
for edge in edges: 
    edge_dict[edge] = count;
    count +=1;
    capacity[edge] = int(randrange(gen)/5+1)
    operator[edge] = randrange(operators)
    tt[edge] = randrange(100)
    G.add_edge(edge[0],edge[1], weight = tt[edge])
    cost[edge] = tt[edge]
    z = random()
    if z <= 0.8 and len(fail) < n + 4:
        p = int(random()*10/10.0) + 0.6;
        failure_prob[edge] = min(p,1);
        prob.append(p);
        fail.append(edge)
        K.append(edge)
        demand[edge] = randrange(gen)*100
        lst_prob.append(p)
    else:
        failure_prob[edge] = 0;
        lst_prob.append(0)

oper_list = list(range(operators))
# Each O-D pair will have an alternative dummy path 10 x times than the shortest path 
import networkx as nx
used_cap = dict()
path_cost = dict()
for pair in K:
    unavailable = []
    try:
            path = nx.shortest_path(G, source=pair[0], target=pair[1], weight = 'weight')
            for edge in list(zip(path,path[1:])):
                if pair[0]==edge[0]:
                    path_cost[(pair[0],pair[1])] = cost[edge]
                else:
                    path_cost[(pair[0],pair[1])] = + cost[edge]
           # print(path)
    except nx.NetworkXNoPath as error:
        print(error.args)
        #raise Exception("not exists")
        #print("not exist")    
nd = gen+1
dummy = []
for x,y in list(enumerate(path_cost)):
    lst_prob.append(0);
    lst_prob.append(0);
    edges.append((y[0], nd));
    edges.append((nd ,y[1]));
    dummy.append((y[0], nd))
    dummy.append((nd ,y[1]));
    tt[(y[0],nd)] = 5 * path_cost[y];
    tt[(nd,y[1])] = 5 * path_cost[y];
    failure_prob[(y[0],nd )] = 0;
    failure_prob[(nd ,y[1])] = 0;
    nd += 1;
nodes = list(range(1,nd))
inflow = {node: [] for node in nodes}
outflow = deepcopy(inflow)
for edge in edges:
    inflow[edge[1]] += [edge]
    outflow[edge[0]] += [edge]

# Computation time of L-Shaped 

In [110]:
start = timeit.default_timer()
# Calculating scenario tree
occurencies = list(itertools.product([0, 1], repeat=len(prob)));
outcome =[];
for lst in occurencies: 
    temp = 1;
    z = -1;
    for elem in lst:
        z += 1;
        if elem == 0:
            temp = temp * (1 - prob[z]);
        else: 
            temp = temp * prob[z];
    outcome.append(temp)  
#outcome = [ round(elem, 3) for elem in outcome ]
failed = { m : k for k,m in enumerate(list(fail))}
P = len(outcome)
J = 1 #2*len(oper_list)*P; # x-only inequalities
M = len(oper_list)#+len(oper_list)*P; # x vars
dataA = np.zeros((M))
rowA = np.zeros(M)
colA = np.arange((M))
A = sp.csr_matrix((dataA, (rowA, colA)), shape=(J,M))
b = np.zeros((1,J));
big_M = 99999999;
A_dict = dict();
k = 0;
for i in oper_list:
    A_dict[i] = k;
    k += 1;
# construct dictionaries
p = np.array(outcome);
y_k_dict = dict();
tmp_q = [];
k = 0;
for pair in K:
    for edge in edges:
        tmp_q.append(tt[edge]);
        y_k_dict[edge,pair] = k;
        k += 1;
for edge in edges:
        tmp_q.append(0);
        y_k_dict[2,edge] = k;
        k += 1;
for edge in edges:
        tmp_q.append(0);
        y_k_dict[3,edge] = k;
        k += 1;
# sizing 
L = len(y_k_dict); #y variables
R = 2*len(nodes)*len(K) # flow constr
R += len(edges) # capacity constr
R += 1 # total excess 
R += len(oper_list) # allocated excess 
R += 2*len(oper_list) # total_slack
#R += 2*len(edges) #e,s<=M*b
# number of scenarios
# construct c matrix
c = np.zeros((M,1));
# construct q matrix
q = np.zeros((L,P));
for i in range(L):
    for j in range(P):
        q[i,j] = tmp_q[i];
# construct T matrix 
#T = [ [ [ 0 for i in range(P) ] for j in range(M)] for z in range(R)];
T = np.zeros((R,M,P));
# construct h matrix 
h =  np.zeros((R,P));
# Second level constraints
k = 0;
dataW = np.empty(0)
rowW = np.empty(0)
colW = np.empty(0)
# flow constraints 
for pair in K:
    temp = demand[pair];
    for node in nodes:
        if node == pair[0]:
            rhs = temp;
        elif node == pair[1]:
            rhs = -temp;
        else:
            rhs = 0;
        for s in range(P):
            h[k,s] = rhs;
            h[k+1,s] = -rhs;
        for edge in outflow[node]: 
                dataW = np.append(dataW,1)
                rowW = np.append(rowW,k)
                colW = np.append(colW,y_k_dict[edge,pair])
                dataW = np.append(dataW,-1)
                rowW = np.append(rowW,k+1)
                colW = np.append(colW,y_k_dict[edge,pair])
        for edge in inflow[node]:      
                dataW = np.append(dataW,-1)
                rowW = np.append(rowW,k)
                colW = np.append(colW,y_k_dict[edge,pair])
                dataW = np.append(dataW,1)
                rowW = np.append(rowW,k+1)
                colW = np.append(colW,y_k_dict[edge,pair])
        k += 2;
# capacity constraints
for edge in edges:
    if edge not in dummy:
        dataW = np.append(dataW,-1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[2,edge])
        dataW = np.append(dataW,1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[3,edge])
        for pair in K:
            dataW = np.append(dataW,1)
            rowW = np.append(rowW,k)
            colW = np.append(colW,y_k_dict[edge,pair])
        for s in range(P):
            if failure_prob[edge]>0 and occurencies[s][failed[edge]]==1:
                  h[k,s] = 0;
            else:
                  h[k,s] = capacity[edge];
        k += 1;
# Allocated excess 
for oper in oper_list:
        for edge in edges:
            if edge not in dummy:
                if operator[edge] == oper:
                    dataW = np.append(dataW,1)
                    rowW = np.append(rowW,k)
                    colW = np.append(colW,y_k_dict[2,edge])
        for oper2 in oper_list: 
            if oper2 != oper:
                for s in range(P):
                    T[k,A_dict[oper2],s] = -1;             
        k += 1
# Total excess
for edge in edges:
    if edge not in dummy:
        dataW = np.append(dataW,1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[2,edge])
for s in range(P):
    for oper2 in oper_list: 
            T[k,A_dict[oper2],s] = -1;  
k += 1;
# Total deficit 
for oper in oper_list:
        for edge in edges:
            if edge not in dummy:
                if operator[edge] == oper: 
                    dataW = np.append(dataW,1)
                    rowW = np.append(rowW,k)
                    colW = np.append(colW,y_k_dict[3,edge])
                    dataW = np.append(dataW,-1)
                    rowW = np.append(rowW,k+1)
                    colW = np.append(colW,y_k_dict[3,edge])
        for s in range(P):
             T[k,A_dict[oper],s] = -1;  
             T[k+1,A_dict[oper],s] = 1;  
        k += 2;
W = sp.csr_matrix((dataW, (rowW, colW)), shape=(R,L))
stop0 = timeit.default_timer()

####### L-shaped method #######

m = Model('master')
m.setParam('OutputFlag', 0)
A_2 = deepcopy(A)
A_2._shape = (np.shape(A_2)[0], np.shape(A_2)[1]+1)
x = m.addMVar(shape=np.shape(A_2)[1],lb=np.append(np.zeros((1,np.shape(A_2)[1]-1)),-big_M), vtype=GRB.CONTINUOUS, name="x")
c_null = np.append(c,0)
c_augmented = np.append(c,1)
m.setObjective(c_null @ x , GRB.MINIMIZE)
m.addConstr(A_2 @ x <= b, name="c")
m.setParam("NodeMethod", 2);
m.setParam("Presolve", 0);
m.update()
m.Params.OutputFlag = 0
num_y_constraints = np.shape(W)[0] 
num_y_values = np.shape(W)[1] 
#K = np.shape(p)[0]
# Preparing feasiblity sub-problem sparse matrix
dataf = deepcopy(dataW)
rowf = deepcopy(rowW)
colf = deepcopy(colW)
for i in range(num_y_constraints):
    dataf = np.append(dataf,1)
    rowf = np.append(rowf,i)
    colf = np.append(colf,2 * i  + num_y_values)
    dataf = np.append(dataf,-1)
    rowf = np.append(rowf,i)
    colf = np.append(colf,2 * i+1+ num_y_values)
constraint_k = sp.csr_matrix((dataf, (rowf, colf)), shape=(num_y_constraints, num_y_values + 2 * num_y_constraints))
num_feasibility_cuts = 0;
num_optimality_cuts = 0;
num_iterations = 0;
review = []
while True: 
    num_iterations += 1;
    
    # Solve the master problem    
    if num_optimality_cuts == 0:
        m.setObjective(c_null @ x , GRB.MINIMIZE)
        m.update()
        m.optimize()
        if m.Status!=2:
            break;
    else:
        m.setObjective(c_augmented @ x , GRB.MINIMIZE)
        m.update()
        m.optimize()
        if m.Status!=2:
            break;
    review.append(m.ObjVal)
    x_augmented =  [var.x for var in  m.getVars()];
    x_optimal =  np.array(x_augmented[:len(x_augmented)-1])
    if num_optimality_cuts == 0:
        theta = -big_M;
    else:
        theta = x_augmented[len(x_augmented)-1]
        
    # Check if there is a need for feasibility cuts
    init_r = num_feasibility_cuts;
    for k in range(P):
        m1 = Model('feasibility cut')
        m1.Params.OutputFlag = 0;
        m1.setParam("NodeMethod", 2);
        m1.setParam("Presolve", 0);
        v = m1.addMVar(shape=num_y_values+2*num_y_constraints,lb=0, vtype=GRB.CONTINUOUS, name="v");
        f = np.append(np.zeros(num_y_values),np.ones(2*num_y_constraints));
        m1.setObjective(f @ v , GRB.MINIMIZE);
        m1.addConstr(constraint_k @ v <= h[:,k]-np.dot(T[:,:,k],np.transpose(x_optimal)), name="c1")
        m1.optimize()
        if m1.Status != 2:
            break;
        if m1.ObjVal >0.001:
            # Feasibility cut
            dual = np.array(m1.getAttr("Pi", m1.getConstrs()));
            coef = np.append(np.dot(dual,T[:,:,k]),0);
            bound = np.dot(dual,h[:,k]);
            coef = np.array([round(elem,3) for elem in coef]);
            bound = round(bound,3);
            m.addConstr(coef @ x >= bound);
            m.update();
            num_feasibility_cuts += 1;
            break;
    if init_r == num_feasibility_cuts:
        bound_sum = 0;
        constraint_sum = np.zeros((1,np.shape(c)[0]));
        for k in range(P):
            m2 = Model('optimality cut');
            m2.setParam('OutputFlag', 0);
            m2.setParam("NodeMethod", 2);
            m2.setParam("Presolve", 0);
            y = m2.addMVar(shape=num_y_values,lb=0, vtype=GRB.CONTINUOUS, name="y");
            m2.setObjective(q[:,k] @ y , GRB.MINIMIZE);
            m2.addConstr(W @ y <= h[:,k]-np.dot(T[:,:,k],np.transpose(x_optimal)), name="c2")
            m2.update()
            m2.Params.OutputFlag = 0
            m2.optimize()
            dual = np.array(m2.getAttr("Pi", m2.getConstrs()));
            bound_sum += p[k] * np.dot(dual,h[:,k]);  
            constraint_sum += p[k] * np.dot(dual,T[:,:,k]);
        current_sum = -np.dot(constraint_sum,np.transpose(x_optimal)) + bound_sum; 
        print(m.ObjVal)
        # Stopping criterion
        int_stop = timeit.default_timer()
        if   current_sum.item() <= theta or abs(theta-current_sum.item())/abs(current_sum.item())<=0.1:
            break;
        else:
            # Optimality cut
            constraint_sum = np.append(constraint_sum, 1) ;
            m.addConstr(constraint_sum @ x >= bound_sum)
            num_optimality_cuts += 1;
stop1 = timeit.default_timer()

print("Number of scenarios :", P)
print("L-Shaped time is :", stop1-start)
print(m.objVal)
#print("opt gap of L-Shaped is {} %".format(round(100 * abs(m3.objVal - m.objVal)/m3.objVal,4)))

0.0
-99999999.0
-99999999.0
-99999999.0
-99999999.0
-99999999.0
-99999999.0
3348638.799999999
Number of scenarios : 256
L-Shaped time is : 44.57984258400393
3348638.799999999


# Computational time of DEP 

In [111]:
start = timeit.default_timer()
# Calculating scenario tree
occurencies = list(itertools.product([0, 1], repeat=len(prob)));
outcome =[];
for lst in occurencies: 
    temp = 1;
    z = -1;
    for elem in lst:
        z += 1;
        if elem == 0:
            temp = temp * (1 - prob[z]);
        else: 
            temp = temp * prob[z];
    outcome.append(temp)  
#outcome = [ round(elem, 3) for elem in outcome ]
failed = { m : k for k,m in enumerate(list(fail))}
P = len(outcome)
J = 1 #2*len(oper_list)*P; # x-only inequalities
M = len(oper_list)#+len(oper_list)*P; # x vars
dataA = np.zeros((M))
rowA = np.zeros(M)
colA = np.arange((M))
A = sp.csr_matrix((dataA, (rowA, colA)), shape=(J,M))
b = np.zeros((1,J));
big_M = 9999999999;
A_dict = dict();
k = 0;
for i in oper_list:
    A_dict[i] = k;
    k += 1;
# construct dictionaries
p = np.array(outcome);
y_k_dict = dict();
tmp_q = [];
k = 0;
for pair in K:
    for edge in edges:
        tmp_q.append(tt[edge]);
        y_k_dict[edge,pair] = k;
        k += 1;
for edge in edges:
        tmp_q.append(0);
        y_k_dict[2,edge] = k;
        k += 1;
for edge in edges:
        tmp_q.append(0);
        y_k_dict[3,edge] = k;
        k += 1;
# sizing 
L = len(y_k_dict); #y variables
R = 2*len(nodes)*len(K) # flow constr
R += len(edges) # capacity constr
R += 1 # total excess 
R += len(oper_list) # allocated excess 
R += 2*len(oper_list) # total_slack
#R += 2*len(edges) #e,s<=M*b
# number of scenarios
# construct c matrix
c = np.zeros((M,1));
# construct q matrix
q = np.zeros((L,P));
for i in range(L):
    for j in range(P):
        q[i,j] = tmp_q[i];
# construct T matrix 
#T = [ [ [ 0 for i in range(P) ] for j in range(M)] for z in range(R)];
T = np.zeros((R,M,P));
# construct h matrix 
h =  np.zeros((R,P));
# Second level constraints
k = 0;
dataW = np.empty(0)
rowW = np.empty(0)
colW = np.empty(0)
# flow constraints 
for pair in K:
    temp = demand[pair];
    for node in nodes:
        if node == pair[0]:
            rhs = temp;
        elif node == pair[1]:
            rhs = -temp;
        else:
            rhs = 0;
        for s in range(P):
            h[k,s] = rhs;
            h[k+1,s] = -rhs;
        for edge in outflow[node]: 
                dataW = np.append(dataW,1)
                rowW = np.append(rowW,k)
                colW = np.append(colW,y_k_dict[edge,pair])
                dataW = np.append(dataW,-1)
                rowW = np.append(rowW,k+1)
                colW = np.append(colW,y_k_dict[edge,pair])
        for edge in inflow[node]:      
                dataW = np.append(dataW,-1)
                rowW = np.append(rowW,k)
                colW = np.append(colW,y_k_dict[edge,pair])
                dataW = np.append(dataW,1)
                rowW = np.append(rowW,k+1)
                colW = np.append(colW,y_k_dict[edge,pair])
        k += 2;
# capacity constraints
for edge in edges:
    if edge not in dummy:
        dataW = np.append(dataW,-1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[2,edge])
        dataW = np.append(dataW,1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[3,edge])
        for pair in K:
            dataW = np.append(dataW,1)
            rowW = np.append(rowW,k)
            colW = np.append(colW,y_k_dict[edge,pair])
        for s in range(P):
            if failure_prob[edge]>0 and occurencies[s][failed[edge]]==1:
                  h[k,s] = 0;
            else:
                  h[k,s] = capacity[edge];
        k += 1;
# Allocated excess 
for oper in oper_list:
        for edge in edges:
            if edge not in dummy:
                if operator[edge] == oper:
                    dataW = np.append(dataW,1)
                    rowW = np.append(rowW,k)
                    colW = np.append(colW,y_k_dict[2,edge])
        for oper2 in oper_list: 
            if oper2 != oper:
                for s in range(P):
                    T[k,A_dict[oper2],s] = -1;             
        k += 1
# Total excess
for edge in edges:
    if edge not in dummy:
        dataW = np.append(dataW,1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[2,edge])
for s in range(P):
    for oper2 in oper_list: 
            T[k,A_dict[oper2],s] = -1;  
k += 1;
# Total deficit 
for oper in oper_list:
        for edge in edges:
            if edge not in dummy:
                if operator[edge] == oper: 
                    dataW = np.append(dataW,1)
                    rowW = np.append(rowW,k)
                    colW = np.append(colW,y_k_dict[3,edge])
                    dataW = np.append(dataW,-1)
                    rowW = np.append(rowW,k+1)
                    colW = np.append(colW,y_k_dict[3,edge])
        for s in range(P):
             T[k,A_dict[oper],s] = -1;  
             T[k+1,A_dict[oper],s] = 1;  
        k += 2;
W = sp.csr_matrix((dataW, (rowW, colW)), shape=(R,L))
stop0 = timeit.default_timer()

####### DEP #######

b_2 = deepcopy(b);
c_2 = deepcopy(c)
data_2 = deepcopy(dataA)
row_2 = deepcopy(rowA)
col_2 = deepcopy(colA)
J = np.shape(A)[0]
M = np.shape(A)[1]
R = np.shape(W)[0]
L = np.shape(W)[1]
for k in range(P):
    c_2 = np.append(c_2,p[k] * q[:,k])
    b_2 = np.append(b_2,h[:,k])
    data_2 = np.append(data_2,dataW) 
    row_2 = np.append(row_2,rowW + J + k * R)
    col_2 = np.append(col_2,colW + M + k * L)
    T_k = sp.csr_matrix(T[:,:,k])
    rowT, colT = T_k.nonzero()
    dataT = T_k[rowT,colT]
    data_2 = np.append(data_2,dataT) 
    row_2 = np.append(row_2,rowT + J + k * R)
    col_2 = np.append(col_2,colT)
A_2 = sp.csr_matrix((data_2, (row_2, col_2)), shape = (J + P * R,M + P * L));
m3 = Model('DEP');
x = m3.addMVar(shape = np.shape(c_2)[0],lb = 0, vtype=GRB.CONTINUOUS, name = "x");
m3.setObjective(c_2 @ x , GRB.MINIMIZE);
m3.addConstr(A_2 @ x <= b_2);
m3.update();
m3.optimize();
stop2 = timeit.default_timer()
print("Number of scenarios :", P);
#print("L-Shaped time is :", stop1-start)
print("DEP time is :" , stop2-start);

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 118017 rows, 163844 columns and 702464 nonzeros
Model fingerprint: 0x52bd9f35
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-04, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 77825 rows and 34176 columns
Presolve time: 0.66s
Presolved: 40192 rows, 129668 columns, 370560 nonzeros

Ordering time: 0.03s

Barrier statistics:
 Dense cols : 4
 AA' NZ     : 2.681e+05
 Factor NZ  : 1.239e+06 (roughly 80 MBytes of memory)
 Factor Ops : 4.639e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.46661743e+07  0.00000000e+00  9.68e+04 0.00e+00  1.61e+03

## Computational time of SAA

In [112]:
start = timeit.default_timer()
J = 1 #2*len(oper_list)*P; # x-only inequalities
M = len(oper_list)#+len(oper_list)*P; # x vars
dataA = np.zeros((M))
rowA = np.zeros(M)
colA = np.arange((M))
A = sp.csr_matrix((dataA, (rowA, colA)), shape=(J,M))
b = np.zeros((1,J));
big_M = 9999999999;
A_dict = dict();
k = 0;
for i in oper_list:
    A_dict[i] = k;
    k += 1;
# construct dictionaries
y_k_dict = dict();
tmp_q = [];
k = 0;
for pair in K:
    for edge in edges:
        tmp_q.append(tt[edge]);
        y_k_dict[edge,pair] = k;
        k += 1;
for edge in edges:
        tmp_q.append(0);
        y_k_dict[2,edge] = k;
        k += 1;
for edge in edges:
        tmp_q.append(0);
        y_k_dict[3,edge] = k;
        k += 1;
# sizing 
L = len(y_k_dict); #y variables
R = 2*len(nodes)*len(K) # flow constr
R += len(edges) # capacity constr
R += 1 # total excess 
R += len(oper_list) # allocated excess 
R += 2*len(oper_list) # total_slack
#R += 2*len(edges) #e,s<=M*b
# number of scenarios
# construct c matrix
c = np.zeros((M,1));
# construct q matrix
#q = np.zeros((L,P));
#for i in range(L):
 #   for j in range(P):
  #      q[i,j] = tmp_q[i];
# construct T matrix 
#T = [ [ [ 0 for i in range(P) ] for j in range(M)] for z in range(R)];
T = np.zeros((R,M));
# construct h matrix 
h =  np.zeros((R));
# Second level constraints
k = 0;
dataW = np.empty(0)
rowW = np.empty(0)
colW = np.empty(0)
# flow constraints 
for pair in K:
    temp = demand[pair];
    for node in nodes:
        if node == pair[0]:
            rhs = temp;
        elif node == pair[1]:
            rhs = -temp;
        else:
            rhs = 0;
        h[k] = rhs;
        h[k+1] = -rhs;
        for edge in outflow[node]: 
                dataW = np.append(dataW,1)
                rowW = np.append(rowW,k)
                colW = np.append(colW,y_k_dict[edge,pair])
                dataW = np.append(dataW,-1)
                rowW = np.append(rowW,k+1)
                colW = np.append(colW,y_k_dict[edge,pair])
        for edge in inflow[node]:      
                dataW = np.append(dataW,-1)
                rowW = np.append(rowW,k)
                colW = np.append(colW,y_k_dict[edge,pair])
                dataW = np.append(dataW,1)
                rowW = np.append(rowW,k+1)
                colW = np.append(colW,y_k_dict[edge,pair])
        k += 2;
# capacity constraints
constr_rows = []
for edge in edges:
    if edge not in dummy:
        constr_rows.append(k)
        dataW = np.append(dataW,-1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[2,edge])
        dataW = np.append(dataW,1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[3,edge])
        h[k] = capacity[edge];
        for pair in K:
            dataW = np.append(dataW,1)
            rowW = np.append(rowW,k)
            colW = np.append(colW,y_k_dict[edge,pair])
        k += 1;
    #for s in range(P):
        #if failure_prob[edge]>0 and occurencies[s][failed[edge]]==1:
             # h[k,s] = 0;
        #else:
              #h[k,s] = capacity[edge];
            
# Allocated excess 
for oper in oper_list:
        for edge in edges:
            if edge not in dummy:
                if operator[edge] == oper:
                    dataW = np.append(dataW,1)
                    rowW = np.append(rowW,k)
                    colW = np.append(colW,y_k_dict[2,edge])
        for oper2 in oper_list: 
            if oper2 != oper:
                T[k,A_dict[oper2]] = -1;             
        k += 1
# Total excess
for edge in edges:
    if edge not in dummy:
        dataW = np.append(dataW,1)
        rowW = np.append(rowW,k)
        colW = np.append(colW,y_k_dict[2,edge])
for oper2 in oper_list: 
        T[k,A_dict[oper2]] = -1;  
k += 1;
# Total deficit 
for oper in oper_list:
        for edge in edges:
            if edge not in dummy:
                if operator[edge] == oper: 
                    dataW = np.append(dataW,1)
                    rowW = np.append(rowW,k)
                    colW = np.append(colW,y_k_dict[3,edge])
                    dataW = np.append(dataW,-1)
                    rowW = np.append(rowW,k+1)
                    colW = np.append(colW,y_k_dict[3,edge])
        #for s in range(P):
        T[k,A_dict[oper]] = -1;  
        T[k+1,A_dict[oper]] = 1;  
        k += 2;
W = sp.csr_matrix((dataW, (rowW, colW)), shape=(R,L))
J = np.shape(A)[0]
M = np.shape(A)[1]
R = np.shape(W)[0]
L = np.shape(W)[1]
q = np.array(tmp_q)
b_2 = deepcopy(b);
c_2 = deepcopy(c);
data_2 = deepcopy(dataA)
row_2 = deepcopy(rowA)
col_2 = deepcopy(colA)
lst_prob = np.array(deepcopy(lst_prob))
count = -1
p_sample = np.zeros((len(lst_prob),))
rtol = 5e-02
atol = 1e-02
cap_begin = constr_rows[0]
while count<=98: #(np.allclose(p_sample, lst_prob,rtol,atol) == False):
    count +=1
    h_trial = deepcopy(h)
    # bernouli trial
    occur = np.random.binomial(1, p=lst_prob)
    for elem in np.nonzero(occur)[0]:
          h_trial[elem+cap_begin] = 0;
    p_sample = (occur+(count)*p_sample)/(count+1)
    c_2 = np.append(c_2,1.0*q)
    b_2 = np.append(b_2,h_trial)
    data_2 = np.append(data_2,dataW) 
    row_2 = np.append(row_2,rowW + J + count * R)
    col_2 = np.append(col_2,colW + M + count * L)
    T_k = sp.csr_matrix(T)
    rowT, colT = T_k.nonzero()
    dataT = T_k[rowT,colT]
    data_2 = np.append(data_2,dataT) 
    row_2 = np.append(row_2,rowT + J + count * R)
    col_2 = np.append(col_2,colT)
A_3 = sp.csr_matrix((data_2, (row_2, col_2)), shape=(J + (count+1)*R, M + (count+1)*L))
m4 = Model('Monte');
c_2 = c_2.astype(float)
c_2[M:] = c_2[M:] / (count+1)
x = m4.addMVar(shape = np.shape(c_2)[0],lb=0, vtype=GRB.CONTINUOUS, name="x");
m4.update()
m4.setObjective(c_2 @ x , GRB.MINIMIZE);
m4.addConstr(A_3 @ x <= b_2)
m4.update()
m4.optimize()
stop3 = timeit.default_timer()
print("Network with pairs :", len(K))
print("SAA samples :", count+1)
print("Network with nodes :", gen)
print("SAA time is :", stop3 - start)
print("opt gap of SAA is {} %".format(round(100*abs(m.objVal - m4.objVal)/m.objVal,4)))

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 46101 rows, 64004 columns and 274400 nonzeros
Model fingerprint: 0x68ed4a8f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 30489 rows and 13498 columns
Presolve time: 0.26s
Presolved: 15612 rows, 50506 columns, 145098 nonzeros

Ordering time: 0.02s

Barrier statistics:
 Dense cols : 4
 AA' NZ     : 1.043e+05
 Factor NZ  : 4.826e+05 (roughly 30 MBytes of memory)
 Factor Ops : 1.793e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.46495768e+07  0.00000000e+00  9.02e+04 0.00e+00  2.36e+03   

In [121]:
# Store parameters 
import pickle
scenarios = 2**len(prob)
pickle.dump( tt, open( "cost_{}.p".format(scenarios), "wb" ) )
pickle.dump( demand, open( "demand_{}.p".format(scenarios), "wb" ) )
pickle.dump( lst_prob, open( "failures_{}.p".format(scenarios), "wb" ) )
pickle.dump( operator, open( "operators_{}.p".format(scenarios), "wb" ) )
pickle.dump( capacity, open( "capacity_{}.p".format(scenarios), "wb" ) )
pickle.dump( prob, open( "only_failed_{}.p".format(scenarios), "wb" ) )
pickle.dump( K, open( "OD_pair_{}.p".format(scenarios), "wb" ) )